In [1]:
'''
In this exercise we will simulate real-time data analysis and feedback on a live acquisition system.
Since we don’t have a real microscope, this example is a bit contrived, but many of the principles its tests apply to the systems we use in the lab daily.

Launch MicroManager as in the previous exercise. Set the camera in “Noise” mode by opening the Device Property Browser and setting the “Mode” property of the “Camera” device to “Noise”.
Instead of the wave pattern you will now see images like this: [Img]

In a new python script, set up a timelapse acquisition with 100 time points in the DAPI channel with exposure time of 10 ms.
Extend your acquisition pipeline to inspect each image as it is acquired. If a pixel has a value of exactly 700 counts,
set neighboring pixels within a radius of 15 pixels to zero. Save this modified image in the .tif dataset instead of the original image.

A modified image may look something like this: [Img]
'''


'\nIn this exercise we will simulate real-time data analysis and feedback on a live acquisition system.\nSince we don’t have a real microscope, this example is a bit contrived, but many of the principles its tests apply to the systems we use in the lab daily.\n\nLaunch MicroManager as in the previous exercise. Set the camera in “Noise” mode by opening the Device Property Browser and setting the “Mode” property of the “Camera” device to “Noise”.\nInstead of the wave pattern you will now see images like this: [Img]\n\nIn a new python script, set up a timelapse acquisition with 100 time points in the DAPI channel with exposure time of 10 ms.\nExtend your acquisition pipeline to inspect each image as it is acquired. If a pixel has a value of exactly 700 counts,\nset neighboring pixels within a radius of 15 pixels to zero. Save this modified image in the .tif dataset instead of the original image.\n\nA modified image may look something like this: [Img]\n'

In [13]:
!pip install scikit-image
from pycromanager import Acquisition, multi_d_acquisition_events
import numpy as np
from skimage import draw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 18.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 16.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 18.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.1 MB/s eta 0:00:00


In [15]:
# Set the parameters
num_time_points = 100
exposure_time_ms = 10
channels = ['DAPI']


In [16]:
# The directory where the data will be saved
save_directory = 'C:/Users/chang/Downloads/2024_AssocRDEng_TakeHome/Exercise2'
acquisition_name = 'my_acquisition'

#Test
save_directory = '/Users/changboyen/Downloads/2024_AssocRDEng_TakeHome/Exercise2'

In [20]:
#Generate Multi_d_acquisition events
'''
Pycromanager Using multi_d_acquisition_events was inferred from https://pycro-manager.readthedocs.io/en/latest/apis.html#pycromanager.multi_d_acquisition_events
'''

events = multi_d_acquisition_events(
    num_time_points=num_time_points,
    channels=channels,
    channel_exposures_ms=[exposure_time_ms],
    order='tpcz'
)

## This acquisition does not require adjusting the x and y positions

In [26]:
# "Extend the acquisition pipeline to inspect each image as it is acquired.“
'''
#   This code block was inspired by the official example at
#   https://github.com/micro-manager/pycro-manager/blob/main/tutorial/Pycro-Manager%20acquisitions%20tutorial.ipynb

#   The concept of the "drawing circle and set value=0 was inspired by
    1. np.where: https://stackoverflow.com/questions/69562075/get-coordinates-with-np-where-in-python
    2. draw.circle: https://devdoc.net/python/scikit-image-doc-0.13.1/api/skimage.draw.html#skimage.draw.circle

'''

with Acquisition(directory=save_directory, name= acquisition_name) as acq:
    for event in events:
        # Get current Images in each event
        image = acq.acquire_image(event)

        # Processing Images: Locate the position where the pixels = 700
        y_coords, x_coords = np.where(image == 700) # Extract the y and x coordinates of pixels with value 700
        position700 = zip(y_coords, x_coords)       # Combine the y and x coordinates into pairs

        # Traverse all the pixels, and set the pixel value to 0 if it is located within a circular area with a radius of 15.
        for y, x in position700:
            rr, cc = draw.circle(y, x, 15, shape=image.shape) 
            image[rr, cc] = 0  # Set those pixels as 0

        # Save Modified Image
        acq.save_image(image)


# When the context manager exits the inspect is complete and the image is saved
print(f"The image saved to {save_directory}/{acquisition_name}")

Exception: Incorrect arguments. 
Expected java.lang.String, java.lang.String, boolean, boolean, int, int, int, int, boolean 
Got <class 'str'>, <class 'str'>, <class 'bool'>, <class 'int'>, <class 'bool'>